In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity
def test_factorability(data):
    print("Kaiser-Meyer-Olkin (KMO) Test:")
    kmo_all,kmo_model=calculate_kmo(data)
    print(kmo_model)
    # performing Bartlett's test
    chi_square, p_value = calculate_bartlett_sphericity(data)
    print("Bartlett's test of sphericity:")
    #checks whether or not the observed variables intercorrelate at all using the observed correlation matrix against the identity matrix.
    print("Chi-square:", chi_square)
    print("P-value:", p_value)

In [ ]:
def parallel_analysis(data, k=20, method="ml", return_ev=False):
    
    ## adapted from https://github.com/connectomicslab/hcp-behavioral-domains
    """Parallel analysis to get the upper limit of how many factors to extract in the factor analysis.

    Parameters
    ----------
    data : DataFrame
        Behavioral data with subjects as rows and variables as columns
    k : int, optional
        How many random datasets to generate, by default 20
    method : str, optional
        Which factor extraction method to use, see factor_analyzer docs for details, by default "ml"
    return_ev : bool, optional
        Whether to return the eigenvalues from the factor analysis and the average eigenvalues from the parallel analysis, by default False

    Returns
    -------
    int(, int, int) 
        The number of factor suggested by parallel analysis and optionally the eigenvalues and average eigenvalues
    """    
    import numpy as np
    import matplotlib.pyplot as plt
    from factor_analyzer import FactorAnalyzer

    # get shape of the data
    n, m = data.shape

    # initialize FactorAnalyzer
    fa = FactorAnalyzer(n_factors=m, rotation="oblimin", method=method)

    # list to store eigenvalues
    eig = np.ones((k, m))

    # loop for k iterations
    for i in range(k):
        # print("Iteration", i+1)
        # generate random data
        rnd_data = np.random.normal(size=(n, m))
        # run factor analysis
        fa.fit(rnd_data)
        # extract eigenvalues
        ev, v = fa.get_eigenvalues()
        eig[i] = eig[i] * ev

    # average eigenvalues for random data
    avg_eig = np.mean(eig, axis=0)

    # run factor analysis on data
    fa.fit(data)
    ev, v = fa.get_eigenvalues()

    # determine suggested no. of factors
    suggestedFactors = sum((ev - avg_eig) > 0)
    if return_ev:
        return suggestedFactors, ev, avg_eig
    else:
        return suggestedFactors

In [ ]:
def screeplot(data):  
    n_factors = data.shape[1]

    fa = FactorAnalyzer(n_factors, rotation=rotat, method=method)
    
    fa.fit(data)
    
    ev, v = fa.get_eigenvalues()
    x = range(1, len(v)+1)
    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(x, ev, 'o-', linewidth=2)
    plt.xlabel('Factors')
    plt.ylabel('Eigenvalue')
    plt.title(f'Scree Plot_{output_prefix}')
    plt.xticks(x)
    plt.show()
    
    num_factors = sum(ev > 1)
    print(f"Ideal number of factors for {output_prefix}: {num_factors}")
        # Save plot to file
    fig.savefig(f'{output_prefix}_EFA_plot.png')
    return num_factors

In [14]:
from factor_analyzer import FactorAnalyzer

def efa_analysis(data,n_factors, method, rotat, output_prefix, save):
    fa = FactorAnalyzer(num_factors, rotation=rotat, method=method)
    fa.fit(data)
    ev, v = fa.get_eigenvalues()
    loadings = fa.loadings_
    threshold=0.3
    loadings[abs(loadings) < threshold] = 0
    # Calculate communalities
    communality = fa.get_communalities()
    factornames = []
    for i in range(n_factors):
        factor = i+1
        factornames.append("factor"+str(factor))
    L = pd.DataFrame(fa.loadings_, index=data.columns, columns=factornames)
    L.loc['Eigenvalues'] = ev[:n_factors]
    L.loc['Communality'] = communality[:n_factors]
    if save:
        with pd.ExcelWriter(f'{output_prefix}_EFA_results.xlsx', mode="w") as writer:
            L.to_excel(writer, sheet_name='Factor Loadings', index=True)
    return fa